In [130]:
import pandas as pd
pd.set_option('display.max_rows', None)


In [131]:
wr_df = pd.read_parquet('dfs/worldrecords.parquet')
wr_df_slim = wr_df[['track','cc','character','vehicle','tires','glider']]
wr_df_slim.character.unique()

array(['Roy', 'Wario', 'Waluigi', 'Baby Daisy', 'Heavy Mii',
       'Heavy Mii Crossing', 'Donkey Kong', 'Champion Link', 'Rosalina',
       'Dry Bowser', 'Blue Yoshi', 'Baby Peach', 'Heavy Mii Bowser',
       'King Boo', 'Link', 'Lemmy', 'Heavy Mii Pac-Man', 'Heavy Mii DK',
       'Bowser', 'Koopa Troopa', 'Black Yoshi'], dtype=object)

In [132]:
character_stats = pd.read_parquet('dfs/character_stats_adjusted.parquet')
kart_stats = pd.read_parquet('dfs/kart_stats.parquet')
tires_stats = pd.read_parquet('dfs/tires_stats.parquet')
glider_stats = pd.read_parquet('dfs/glider_stats.parquet')
character_stats

,name,speed,acceleration,weight,handling,traction/grip,mini-turbo
0,Baby Peach,2.25,4.00,2.00,5.00,4.25,4.00
1,Baby Daisy,2.25,4.00,2.00,5.00,4.25,4.00
2,Baby Rosalina,2.25,4.25,2.00,4.75,3.75,4.00
3,Lemmy,2.25,4.25,2.00,4.75,3.75,4.00
4,Baby Mario,2.50,4.25,2.00,4.50,4.00,3.75
5,Baby Luigi,2.50,4.25,2.00,4.50,4.00,3.75
6,Dry Bones,2.50,4.25,2.00,4.50,4.00,3.75
7,Koopa Troopa,2.75,4.00,2.50,4.50,4.25,3.75
8,Lakitu,2.75,4.00,2.50,4.50,4.25,3.75
9,Bowser Jr.,2.75,4.00,2.50,4.50,4.25,3.75


In [133]:
# get list of stats columns and append them to the world record dataframe
for column in glider_stats.columns:
    if column != 'name':
        wr_df_slim[column] = 0
# wr_df_slim

/var/folders/d8/v4dr08kj4k5c9zv_bg2sv47r0000gn/T/ipykernel_62250/1215767457.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wr_df_slim[column] = 0
/var/folders/d8/v4dr08kj4k5c9zv_bg2sv47r0000gn/T/ipykernel_62250/1215767457.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wr_df_slim[column] = 0
/var/folders/d8/v4dr08kj4k5c9zv_bg2sv47r0000gn/T/ipykernel_62250/1215767457.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_index

In [134]:
kart_name_conversion = {
    'W 25 Silver Arrow': 'Mercedes Silver Arrow',
    'Gold Kart': 'Gold Standard'
}
kart_convert = lambda kart_name: kart_name_conversion[kart_name] if kart_name in kart_name_conversion else kart_name

In [136]:
def add_data(row):
    tire_data = tires_stats[tires_stats['tire name'] == row.tires.strip()].drop(['tire name','total'], axis=1).to_numpy()[0]
    kart_name = kart_convert(row.vehicle.strip())
    kart_data = kart_stats[kart_stats['kart part'] == kart_name].drop(['kart part','total'], axis=1).to_numpy()[0]
    glider_data = glider_stats[glider_stats['name'] == row.glider.strip()].drop(['name'], axis=1).to_numpy()[0]
    # print(row.character)

    character_data = character_stats[character_stats['name'] == row.character.strip()].drop('name',axis=1).to_numpy()[0]
    # print(character_data)
    row[['speed','acceleration','weight','handling','traction/grip','mini-turbo']] += kart_data + tire_data + glider_data +character_data
    return row
wr_df_slim.apply(add_data,axis=1)

,track,cc,character,vehicle,tires,glider,speed,acceleration,weight,handling,traction/grip,mini-turbo
0,Mario Kart Stadium,150,Roy,Wild Wiggler,Azure Roller,Paper Glider,3.50,4.25,3.00,3.50,3.25,4.25
1,Mario Kart Stadium,200,Wario,Wild Wiggler,Azure Roller,Paper Glider,3.75,4.00,3.25,3.25,3.50,4.00
2,Water Park,150,Waluigi,Mr. Scooty,Azure Roller,Paper Glider,3.00,4.75,2.75,3.75,3.00,4.75
3,Water Park,200,Wario,Biddybuggy,Azure Roller,Paper Glider,3.25,4.50,3.00,3.50,3.50,4.00
4,Sweet Sweet Canyon,150,Baby Daisy,Biddybuggy,Azure Roller,Parachute,0.75,5.50,0.75,5.75,4.50,5.25
5,Sweet Sweet Canyon,200,Wario,Biddybuggy,Button,Flower Glider,3.50,4.25,3.00,3.25,3.25,3.75
6,Thwomp Ruins,150,Baby Daisy,Biddybuggy,Azure Roller,Paper Glider,0.75,5.50,0.75,5.75,4.50,5.25
7,Thwomp Ruins,200,Wario,W 25 Silver Arrow,Azure Roller,Paper Glider,3.75,4.00,3.25,3.25,3.50,4.00
8,Mario Circuit,150,Waluigi,Biddybuggy,Leaf Tires,Paper Glider,3.25,4.50,2.75,3.50,3.00,4.00
9,Mario Circuit,200,Heavy Mii,W 25 Silver Arrow,Leaf Tires,Paper Glider,3.50,3.00,3.25,3.00,3.50,2.75
